In [8]:
import sys
sys.path.append('../../')

from IPython.core.display import display
import numpy as np
import pandas as pd
import networkx as nx
import utils.graph_utils as graph_utils
from utils.IO_utils import load_ground_true
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import svm


In [9]:
# settings
np.random.seed(2020)
name = "Dblp"
dim = 128
come_model_type = "GMM"
ks = [20]

In [10]:
# import graph and true labels

# graph
G = graph_utils.load_matfile(f"../../data/{name}/{name}.mat", undirected=True)

# labels_true
labels_true, _ = load_ground_true(path=f"../../data/{name}", file_name=name)
labels_true = np.array(labels_true) - 1  # 1..5 -> 0..4

In [11]:
# import node embeddings and predicted labels
dfs = {}
for k in ks:
    # import node embeddings
    df = pd.read_csv(
        f"./data/{name}_alpha-0.1_beta-0.1_ws-10_neg-5_lr-0.025_icom-219_ind-219_ds-0.0_d-{dim}_type-{come_model_type}_k-{k}.txt",
        sep="\t| ",
        header=None
    )
    df = df.rename(columns={0: 'node'})
    df.set_index(['node'], inplace=True)

    # import predicted labels
    labels_pred = pd.read_csv(f"./data/labels_pred_{come_model_type}_d{dim}_k{k}.txt", header=None)
    labels_pred = labels_pred.rename(columns={0: 'label_pred'})
    labels_pred.label_pred = labels_pred.astype(int)

    # join labels pred and true to embeddings
    df = df.join(labels_pred)
    df['label_true'] = labels_true

    print(f"k = {k}")
    display(df)
    display(df.describe())
    dfs[k] = df

/Users/anton/miniconda3/envs/MT/lib/python3.6/site-packages/ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


k = 20


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
node,,,,,,,,,,,,,,,,,,,,,
0,0.436527,-0.564456,-0.454166,0.641066,0.168700,0.082310,-0.031954,0.361073,-0.208672,0.276056,...,0.050538,-0.224988,-0.431624,0.003662,-0.477081,0.121960,0.052909,-0.350542,1,0
1,0.403277,-0.511639,-0.333275,0.545582,0.051169,0.028561,-0.091227,0.370079,-0.289292,0.483094,...,-0.015528,-0.363553,-0.475505,0.014565,-0.530358,-0.040160,0.263523,-0.140697,1,2
2,0.487985,-0.513284,-0.554191,0.689418,0.271111,0.052279,-0.025626,0.387778,-0.092199,0.317105,...,-0.009474,-0.074029,-0.445090,-0.088949,-0.461879,0.054054,0.101626,-0.306905,1,2
3,0.424656,-0.452077,-0.523225,0.574095,0.080523,-0.010256,0.022688,0.317095,-0.171146,0.266117,...,-0.084191,-0.234062,-0.329075,-0.094375,-0.241655,0.048418,0.168131,-0.347555,1,0
4,0.372900,-0.446677,-0.523795,0.631834,0.126949,0.054397,-0.117205,0.325753,-0.081025,0.227695,...,0.068887,-0.225661,-0.478980,-0.154673,-0.311928,-0.065331,0.082394,-0.291411,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13179,0.374316,-0.279774,-0.335749,0.397761,0.261890,-0.008758,-0.188682,0.413393,-0.071640,0.283936,...,-0.216151,-0.344590,-0.358659,-0.285082,-0.202561,-0.054232,0.287505,-0.183615,1,0
13180,0.054672,-0.411832,-0.357445,-0.168166,0.181663,0.170066,-0.083065,0.187582,0.377385,0.344669,...,-0.356740,0.299442,-0.133717,0.038592,0.229791,-0.662603,0.141447,0.282151,8,3
13181,-0.020217,-0.474941,-0.114838,0.143250,0.312997,-0.027986,-0.050794,-0.158910,0.536100,0.444589,...,0.131581,0.298015,-0.208109,0.016640,-0.277913,-0.598272,0.087692,0.395714,10,3


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
count,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,...,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000
mean,0.000830,-0.270871,-0.011392,0.247383,0.258536,-0.116252,0.136543,0.129188,0.110883,0.370419,...,-0.133110,0.136703,-0.151722,-0.037213,-0.070353,-0.016308,0.086585,0.091487,8.838365,2.190382
std,0.189273,0.221827,0.219088,0.210020,0.186759,0.217118,0.205720,0.212195,0.215034,0.175124,...,0.177751,0.208749,0.211260,0.179676,0.208271,0.186768,0.217748,0.217452,5.582605,1.315339
min,-0.760121,-1.043094,-0.790480,-0.482115,-0.385856,-0.820103,-0.522908,-0.544886,-0.563251,-0.230095,...,-0.751202,-0.681993,-0.866390,-0.900184,-0.804824,-0.864746,-0.581872,-0.646253,0.000000,0.000000
25%,-0.125075,-0.430068,-0.155517,0.103405,0.134624,-0.276165,-0.008557,-0.012369,-0.048370,0.253831,...,-0.248624,-0.003494,-0.301003,-0.141389,-0.221836,-0.136753,-0.066092,-0.063615,4.000000,1.000000
50%,-0.005971,-0.275395,-0.017926,0.252129,0.252892,-0.121410,0.134579,0.120678,0.090903,0.374856,...,-0.129734,0.141454,-0.148097,-0.028531,-0.077440,-0.014440,0.067656,0.097432,9.000000,3.000000
75%,0.132399,-0.114734,0.127231,0.389884,0.372108,0.028221,0.280896,0.264553,0.252303,0.487054,...,-0.011120,0.280569,-0.007655,0.077459,0.069192,0.108009,0.215292,0.249464,14.000000,3.000000
max,0.701677,0.452445,0.678930,1.098140,0.925440,0.663360,0.771111,0.954931,0.978782,1.002611,...,0.470287,0.805955,0.601112,0.714436,0.653253,0.684004,0.912726,0.733946,19.000000,4.000000


# Community Detection

In [12]:
# Conductance

for k in dfs:
    print("K =", k)
    df = dfs[k]
    conductance_min = 1
    for i in range(k):
        nodes = df[df.label_pred == i].index.to_numpy()
        if len(nodes) == 0: continue
        conductance_i = nx.conductance(G, nodes)
        print(f"  i{i} conductance: {conductance_i}")
        conductance_min = min(conductance_min, conductance_i)
    print(f"=>conductance: {conductance_min}")

K = 20
  i0 conductance: 0.4314936318023929
  i1 conductance: 0.28
  i2 conductance: 0.4953125
  i3 conductance: 0.4541284403669725
  i4 conductance: 0.3968934118907338
  i5 conductance: 0.5799713876967096
  i6 conductance: 0.2177379172894868
  i7 conductance: 0.4944215191808039
  i8 conductance: 0.275626423690205
  i9 conductance: 0.11969825263644061
  i10 conductance: 0.4384917943849179
  i11 conductance: 0.6968048946295038
  i12 conductance: 0.42738950576964946
  i13 conductance: 0.5566807313642756
  i14 conductance: 0.49779278109581926
  i15 conductance: 0.5639548283014519
  i16 conductance: 0.44936708860759494
  i17 conductance: 0.5272815651962571
  i18 conductance: 0.5809213208316347
  i19 conductance: 0.5861451179456554
=>conductance: 0.11969825263644061


In [13]:
# NMI

for k in dfs:
    print("K =", k)
    df = dfs[k]
    labels_pred = df.label_pred.to_numpy()
    labels_true = df.label_true.to_numpy()

    nmi = metrics.normalized_mutual_info_score(labels_true, labels_pred)
    print("NMI: ", nmi)

K = 20
NMI:  0.3018911505699723


# Node Classification

In [14]:
# train LibSVM classifier on 90% of data and test on 10%.

for k in ks:
    print(f"k = {k}")
    df = dfs[k]

    # train/test split
    train, test = train_test_split(df, test_size=0.1)
    feature_col = np.arange(0, dim) + 1
    X_train = train.loc[:, feature_col]
    Y_train = train.label_true
    X_test = test.loc[:, feature_col]
    Y_test = test.label_true

    # fit
    clf = svm.SVC()
    clf.fit(X_train, Y_train)

    # pred
    Y_pred = clf.predict(X_test)

    # results
    micro_f1 = metrics.f1_score(Y_test, Y_pred, average='micro')
    macro_f1 = metrics.f1_score(Y_test, Y_pred, average='macro')
    print(f"  micro_f1: {micro_f1*100:.2f}")
    print(f"  macro_f1: {macro_f1*100:.2f}")

k = 20
  micro_f1: 92.65
  macro_f1: 92.66
